In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/plant/plant_123.csv


In [4]:
df=pd.read_csv("/kaggle/input/plant/plant_123.csv")
df.head()

,Unnamed: 0,common_name,family,categories,origin,climate,zone,img_url
0,0,Janet Craig,Liliaceae,Dracaena,Cultivar,Tropical,"[11,10]",http://www.tropicopia.com/house-plant/thumbnai...
1,1,Lady palm,Arecaceae,Palm,China,Subtropical,"[11, 9]",http://www.tropicopia.com/house-plant/thumbnai...
2,2,"Tailflower , Wax flower",Araceae,Anthurium,Cultivar,Tropical,[11],http://www.tropicopia.com/house-plant/thumbnai...
3,3,Elephant ear,Araceae,Other,Hybrid,Tropical humid,[11],http://www.tropicopia.com/house-plant/thumbnai...
4,4,Malaysian Dracaena,Liliaceae,Dracaena,Cultivar,Tropical,"[11,10]",http://www.tropicopia.com/house-plant/thumbnai...


In [5]:
df.isnull().sum()

Unnamed: 0      0
common_name    21
family          0
categories      0
origin          2
climate         0
zone            0
img_url         0
dtype: int64

In [6]:
for index, row in df.iterrows():
    if pd.isnull(row['common_name']):
        correlated_columns = ['family', 'categories', 'origin']
        max_corr_col = max(correlated_columns, key=lambda col: row[col] if not pd.isnull(row[col]) else 0)
        df.at[index, 'common_name'] = row[max_corr_col]

# Display the updated dataframe
print(df)


     Unnamed: 0                      common_name          family  \
0             0                    Janet Craig         Liliaceae   
1             1                      Lady palm         Arecaceae   
2             2       Tailflower ,  Wax flower           Araceae   
3             3                   Elephant ear           Araceae   
4             4             Malaysian Dracaena         Liliaceae   
..          ...                              ...             ...   
350         350              Chinese Evergreen           Araceae   
351         351              Chinese Evergreen           Araceae   
352         352                Silver Calathea       Marantaceae   
353         353                   Rubber plant          Moraceae   
354         354    Blue Agave ,  Century plant    Amaryllidaceae   

             categories        origin         climate     zone  \
0              Dracaena    Cultivar          Tropical  [11,10]   
1                  Palm       China       Subtropic

In [7]:
df.isnull().sum()

Unnamed: 0     0
common_name    0
family         0
categories     0
origin         2
climate        0
zone           0
img_url        0
dtype: int64

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming your dataframe is named df
# Fill NaN values in 'origin' column with the most correlated non-null value from other columns
for index, row in df.iterrows():
    if pd.isnull(row['origin']):
        correlated_columns = ['family', 'categories', 'common_name']
        
        # Filter valid columns with non-null values
        valid_columns = [col for col in correlated_columns if not pd.isnull(row[col]) and not df[col].isnull().all()]
        
        # If there are no valid columns, skip
        if not valid_columns:
            continue
        
        # Convert valid columns to a single string using one-hot encoding
        one_hot_encoded = pd.get_dummies(df[valid_columns].apply(lambda x: ' '.join(str(val) for val in x), axis=1))
        text = ' '.join(str(val) for val in one_hot_encoded.loc[index])
        
        # Use TfidfVectorizer to calculate similarity
        tfidf = TfidfVectorizer().fit_transform([text] + list(one_hot_encoded.apply(lambda x: ' '.join(str(val) for val in x), axis=1)))
        similarity = cosine_similarity(tfidf)[0, 1:]
        
        # Find the column with the highest similarity
        max_corr_col = valid_columns[similarity.argmax()]
        
        # Update 'origin' with the value from the most correlated column
        df.at[index, 'origin'] = row[max_corr_col]

# Display the updated dataframe
print(df)
df.isnull().sum()

     Unnamed: 0                      common_name          family  \
0             0                    Janet Craig         Liliaceae   
1             1                      Lady palm         Arecaceae   
2             2       Tailflower ,  Wax flower           Araceae   
3             3                   Elephant ear           Araceae   
4             4             Malaysian Dracaena         Liliaceae   
..          ...                              ...             ...   
350         350              Chinese Evergreen           Araceae   
351         351              Chinese Evergreen           Araceae   
352         352                Silver Calathea       Marantaceae   
353         353                   Rubber plant          Moraceae   
354         354    Blue Agave ,  Century plant    Amaryllidaceae   

             categories        origin         climate     zone  \
0              Dracaena    Cultivar          Tropical  [11,10]   
1                  Palm       China       Subtropic

Unnamed: 0     0
common_name    0
family         0
categories     0
origin         0
climate        0
zone           0
img_url        0
dtype: int64

In [20]:
# Assuming your dataframe is named df
# Drop the 'Unnamed: 0' column
df = df.drop('Unnamed: 0', axis=1)

# Display the updated dataframe
print(df)


                         common_name          family          categories  \
0                      Janet Craig         Liliaceae            Dracaena   
1                        Lady palm         Arecaceae                Palm   
2         Tailflower ,  Wax flower           Araceae           Anthurium   
3                     Elephant ear           Araceae               Other   
4               Malaysian Dracaena         Liliaceae            Dracaena   
..                               ...             ...                 ...   
350              Chinese Evergreen           Araceae           Aglaonema   
351              Chinese Evergreen           Araceae           Aglaonema   
352                Silver Calathea       Marantaceae       Foliage plant   
353                   Rubber plant          Moraceae               Ficus   
354    Blue Agave ,  Century plant    Amaryllidaceae  Cactus & Succulent   

           origin         climate     zone  \
0      Cultivar          Tropical  [11,10

In [34]:
from fastai.tabular.all import *
# If 'zone' column contains Python expressions stored as strings
df['zone'] = df['zone'].apply(lambda x: eval(x) if pd.notnull(x) and isinstance(x, str) else x)

# Define categorical and continuous variables
cat_vars = ['common_name', 'family', 'categories', 'origin']
cont_vars = ['zone']

# Define dependent variable
dep_var = 'climate'

# Create pre-processing blocks for categorical and continuous variables
procs = [Categorify, FillMissing]

# Split the data into training and validation sets
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Create TabularPandas objects for training and validation sets
to = TabularPandas(df, procs, cat_vars, cont_vars, y_names=dep_var, splits=[list(range(len(train_df))), list(range(len(train_df), len(df)))], y_block=CategoryBlock())

# Create a fastai tabular_learner
dls = to.dataloaders(bs=64)
learn = tabular_learner(dls, layers=[200,100], metrics=accuracy)

# Fit the model on the training data
learn.fit_one_cycle(5)

# Make predictions on the validation set
valid_preds = learn.get_preds(dl=to.valid)[0].argmax(dim=-1)

# Evaluate the model
accuracy = (valid_preds == to.valid.ys.values).float().mean()
print(f'Validation Accuracy: {accuracy}')


epoch,train_loss,valid_loss,accuracy,time
0,2.012121,1.843842,0.014085,00:01
1,1.892266,1.819074,0.014085,00:00
2,1.747261,1.778473,0.014085,00:00
3,1.634586,1.762967,0.014085,00:00
4,1.566736,1.774082,0.056338,00:00


TypeError: string indices must be integers

In [32]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Process the 'zone' column to extract numerical information
df['zone'] = df['zone'].apply(lambda x: eval(x)[0] if (isinstance(x, str) and pd.notnull(x)) else x)

# Define categorical and continuous variables
cat_vars = ['common_name', 'family', 'categories', 'origin']
cont_vars = ['zone']

# Define dependent variable
dep_var = 'climate'

# Convert 'climate' to a numerical category for classification
df['climate'] = df['climate'].astype('category').cat.codes

# Create pre-processing blocks for categorical and continuous variables
procs = [Categorify, FillMissing, Normalize]

# Split the data into training and validation sets
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Create TabularPandas objects for training and validation sets
to = TabularPandas(df, procs, cat_vars, cont_vars, y_names=dep_var, splits=[list(range(len(train_df))), list(range(len(train_df), len(df)))], y_block=CategoryBlock())

# Create a RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=4, random_state=42)

# Fit the model on the training data
rf_model.fit(to.train.xs, to.train.ys.values.ravel())

# Make predictions on the validation set
valid_preds = rf_model.predict(to.valid.xs)

# Evaluate the model
accuracy = accuracy_score(to.valid.ys.values.ravel(), valid_preds)
print(f'Validation Accuracy: {accuracy}')


Validation Accuracy: 0.8591549295774648


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, Input, concatenate
from keras.optimizers import Adam

# Load your data
df = pd.read_csv('/kaggle/input/plant/plant_123.csv')

# Preprocess your data
df = df.dropna()  # drop rows with missing values
encoder = OneHotEncoder(sparse=False)
scaler = StandardScaler()
# Convert string representation of list to actual list
import ast
df['zone'] = df['zone'].apply(ast.literal_eval)

# Calculate the mean of the list
df['zone'] = df['zone'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else 0)

# Assuming 'categories', 'climate' are categorical and 'zone' is numerical
categorical_data = encoder.fit_transform(df[['categories', 'climate']])
numerical_data = scaler.fit_transform(df[['zone']])

from PIL import Image
import requests
from io import BytesIO

image_data = []
for img_url in df['img_url']:
    try:
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((224, 224))  # resize image
        img = img_to_array(img)  # convert image to array
        img = preprocess_input(img)  # preprocess image
        image_data.append(img)
    except Exception as e:
        print(f"Could not load image {img_url}. Error: {e}")


# Split data into training and test sets
X_train_cat, X_test_cat, X_train_num, X_test_num, X_train_img, X_test_img, y_train, y_test = train_test_split(
    categorical_data, numerical_data, image_data, df['common_name'], test_size=0.2, random_state=42)

# Define your model
categorical_input = Input(shape=(X_train_cat.shape[1],))
numerical_input = Input(shape=(X_train_num.shape[1],))
image_input = Input(shape=(224, 224, 3))

# Use a pre-trained CNN for image feature extraction
cnn = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
cnn.trainable = False  # freeze the convolutional base

image_features = cnn(image_input)
image_features = Flatten()(image_features)

# Concatenate all features
x = concatenate([categorical_input, numerical_input, image_features])

# Add a dense layer and an output layer
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Create and compile the model
model = Model(inputs=[categorical_input, numerical_input, image_input], outputs=output)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train_cat, X_train_num, X_train_img], y_train, epochs=10, batch_size=32)






/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Data cardinality is ambiguous:
  x sizes: 265, 265, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 2

In [41]:
print(f"Length of X_train_cat: {len(X_train_cat)}")
print(f"Length of X_train_num: {len(X_train_num)}")
print(f"Length of X_train_img: {len(X_train_img)}")
print(f"Length of y_train: {len(y_train)}")


Length of X_train_cat: 265
Length of X_train_num: 265
Length of X_train_img: 265
Length of y_train: 265
